In [35]:
import os
from matplotlib.pyplot import axis
import numpy as np
import pandas as pd

In [36]:
rows = 200
columns = 299
layers = 76

In [37]:
def oneLayerPointsIdxs(layer):
    b = layer*rows*columns # b = begin
    points_indexs = []
    for i in range(rows*(columns-1)):
        points_indexs.append([b+i,b+i+1,b+i+rows,b+i+rows+1])
    points_indexs = np.delete(points_indexs,delete_from_pointsIdx_idx,axis=0)

    return points_indexs

In [38]:
"""
生成中点坐标

坐标排布顺序:    (x2,y2)  (x4,y4)
                (x1,y1)  (x3,y3)   
"""
def crossPoint(x1,y1,x2,y2,x3,y3,x4,y4):
    if x1 == x4:
        k1 = None
        b1 = 0
    else:
        k1 = (y4-y1)/(x4-x1)
        b1 = y1-k1*x1
    if x2 == x3:
        k2 = None
        b2 = 0
    else:
        k2 = (y3-y2)/(x3-x2)
        b2 = y2-k2*x2
    if k1 == None:
        x = x1
        y = k2*x + b2
    elif k2 == None:
        x = x2
        y = k1*x+b1
    # elif k1 == k2:
    #     x = np.nan
    #     y = np.nan
    else:
        x = (b2-b1) / (k1-k2)
        y = k1*x + b1
    return [x,y]

In [61]:
def CenterPoint(points_indexs):
    center_points = []
    for i in points_indexs:
        centers = []
        # 8个相机的xy，和1个世界坐标点的xyz
        for j in range(8+1):
            [x1,y1,x2,y2,x3,y3,x4,y4] = [data[i[0]][0+2*j],data[i[0]][1+2*j],data[i[1]][0+2*j],data[i[1]][1+2*j],data[i[2]][0+2*j],data[i[2]][1+2*j],data[i[3]][0+2*j],data[i[3]][1+2*j]]
            center = crossPoint(x1,y1,x2,y2,x3,y3,x4,y4)
            centers.append(center[0])
            centers.append(center[1])
        # 补充上z轴数据
        centers.append(data[i[0]][18])
        np.ravel(centers)
        center_points.append(centers)
    return center_points

In [40]:
def deleteFromPointsIdx():
    delete_from_pointsIdx_idx= []
    for column in range(1,columns):
        delete_from_pointsIdx_idx.append(rows*column-1)
    return delete_from_pointsIdx_idx

In [41]:
print("reading data")
imgpoints = pd.read_csv("./data/points_cloud.csv")
worldpoints = pd.read_csv("./data/worldpoints.csv")

frames = [imgpoints, worldpoints]
df = pd.concat(frames, axis=1)
data = np.array(df)

reading data


In [62]:
df.columns

Index(['0x', '0y', '1x', '1y', '2x', '2y', '3x', '3y', '4x', '4y', '5x', '5y',
       '6x', '6y', '7x', '7y', 'x', 'y', 'z'],
      dtype='object')

In [63]:
delete_from_pointsIdx_idx = deleteFromPointsIdx()

In [64]:
layer = 0
points_indexs = oneLayerPointsIdxs(layer)

In [65]:
centerpoints = CenterPoint(points_indexs)

In [66]:
pd.DataFrame(centerpoints,columns=df.columns)

,0x,0y,1x,1y,2x,2y,3x,3y,4x,4y,5x,5y,6x,6y,7x,7y,x,y,z
0,1251.806923,2468.620172,1120.067372,2855.200076,898.267931,2696.211108,661.839390,2729.763123,3862.655141,375.033180,3972.205035,460.681516,3855.752386,490.848775,3768.867054,788.075996,1.905,1.905,0.0
1,1250.248899,2460.877354,1118.939084,2846.265758,897.227760,2686.887196,662.889397,2719.799959,3863.304535,387.060329,3971.000724,472.739649,3854.491661,502.003050,3766.883485,797.709840,1.905,5.715,0.0
2,1248.668253,2452.823745,1117.818947,2837.327263,896.250089,2677.589957,663.971743,2710.057905,3863.927142,399.075375,3969.857861,484.749853,3853.380035,512.895148,3765.090332,807.373608,1.905,9.525,0.0
3,1247.063353,2445.233803,1116.663865,2828.565542,895.219708,2668.507731,665.032104,2700.293115,3864.551936,411.044755,3968.730855,496.699403,3852.183080,524.030168,3763.479399,816.915640,1.905,13.335,0.0
4,1245.692002,2437.425277,1115.489725,2819.655021,894.256568,2659.225152,666.038854,2690.551287,3865.188806,422.976492,3967.624866,508.669774,3851.143555,534.913480,3761.750933,826.257316,1.905,17.145,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59297,3873.945552,455.332814,4106.297341,802.818921,3967.152761,615.448864,3753.770040,1125.875885,1282.416867,2411.408420,881.476498,2382.236929,743.687958,2531.045034,753.938934,2597.617905,1133.475,741.045,0.0
59298,3873.649081,442.956822,4106.543411,791.160996,3968.204757,604.249326,3755.820015,1116.726355,1284.134751,2418.954284,882.428137,2390.824134,744.565717,2540.351457,753.699520,2607.321051,1133.475,744.855,0.0
59299,3873.254525,430.596120,4106.675008,779.447720,3969.227562,593.110333,3757.995687,1107.473215,1285.909828,2427.142800,883.369654,2399.649681,745.503436,2549.723130,753.575067,2616.920800,1133.475,748.665,0.0
59300,3872.896916,418.339944,4106.931264,767.762660,3970.361039,581.832927,3760.312844,1098.385167,1287.487334,2434.923592,884.306414,2408.255032,746.415722,2559.031820,753.391060,2626.592820,1133.475,752.475,0.0
